In [2]:
import theano
import theano.tensor as T

In [3]:
class layer:
    def __init__(self, W_init, b_init, activation_func):
        '''
        output = activation(Wx+b)
        x is the input one dimensional vector
        W takes 'n_in' inputs from the previous layer (the number of previous layers nodes)
        and has 'n_out' outputs (the number of the next layers nodes)
        
        parameters:
        1. W_init: initial weight values
            np.ndarray
        2. b_init: initial bias value
            np.ndarray
        3. activation_func: activation function for output of layer
            theano.tensor.elemwise.Elemwise
        '''
        n_out, n_in = W_init.shape
        assert b_init.shape == (n_out,)
        self.W = theano.shared(W_init.astype(theano.config.floatX), shared=True)
        #shared=True as we allow Theano to use memory for this object (make faster)
        self.b = theano.shared(b_init.astype(theano.config.floatX), shared=True)
        self.activation_func = self.activation_func
        self.params = [self.W, self.b]